In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [3]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [4]:
SYSTEM_PROMPT = """
Based on the provided use case specification, please generate a set of test scenarios to thoroughly test the functionality described in the use case.
Your task is to generate suitable test scenarios that cover all the flows and test the boundaries of conditions, if possible. Each scenario should clearly and specifically describe the situation to be tested.
Example for condition: "a valid username must be over 8 characters and below 30 characters." When you meets something like this, make sure to treat each condition as a negative scenario seperately. That means you will have two negative scenario: invalid username with lower than 8 characters and invalid username with higher than 30 characters. 
Please ensure that your test scenarios cover both the main flow and any alternate flows mentioned in the use case specification. Also, consider potential exception flows to address errors, failures, or unexpected events that could occur during the execution of the use case and encompass negative test scenarios (including edge cases and invalid inputs), and unexpected user behaviors.
Keep in mind the agile nature of the application and prioritize scenarios that align closely with the given use case specification and the application's requirements.
Return the list of test scenarios name to test the use case
Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

In [5]:
usecase_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Shopping System"
save_path = r"D:\Dissertation-GPT\evaluate\prove\Shopping System\ts"

In [6]:
usecase_name_list = []
project_name = os.path.basename(usecase_path)

In [7]:
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [8]:
for usecase_name in usecase_name_list:
    print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    prompt_all = read_file_content(usecase_directlink)
    promptSYSTEM_PROMPT = [
        { "role": "system", "content": SYSTEM_PROMPT},
        { "role": "user", "content": prompt_all}
    ]
    gpt_response = ask(promptSYSTEM_PROMPT, client, model)    
    print("gpt_response \n" +gpt_response)
    write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), gpt_response)

add to cart
gpt_response 
1. Successful Item Addition to Cart
2. Successful Item Addition with Option Selection
3. Successful Item Quantity Adjustment Before Adding to Cart
4. Attempt to Add Item Without Selecting Option
5. Attempt to Increase Quantity Beyond Stock Limit
6. Attempt to Decrease Quantity Below Minimum
7. Attempt to Add Out of Stock Item to Cart
8. Attempt to Add Item with Out of Stock Option
9. View Cart After Adding Item
10. Add Multiple Different Items to Cart
11. Add Multiple Quantities of the Same Item to Cart
12. Add Item to Cart with Maximum Stock Quantity
String has been written to D:\Dissertation-GPT\evaluate\prove\Shopping System\ts\add to cart.txt
checkout
gpt_response 
1. Successful Checkout with Default Settings
2. Successful Checkout with Changed Shipping Address
3. Successful Checkout with New Shipping Address
4. Successful Checkout with Changed Payment Method
5. Successful Checkout with Application Voucher Applied
6. Successful Checkout with Shop Voucher A